based on 

https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
from custom_tensorboard import TrainValTensorBoard
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

In [5]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [6]:
# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [7]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [8]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix)
    #trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix)
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [9]:
x_train, y_train, x_test, y_test = load_dataset('../data/HAR/UCI_HAR_Dataset/')

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [10]:
x_train.shape

(7352, 128, 9)

In [11]:
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]

In [12]:
epochs = 100
batch_size = 500

keras imports

In [13]:
from keras import Model
from keras.layers import Lambda, Input, Dropout, Flatten, LSTM, Concatenate
from keras import backend as K
from keras.callbacks import TensorBoard
from time import time
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [23]:
import tensorflow as tf 

def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

auc_roc = as_keras_metric(tf.metrics.auc)
recall = as_keras_metric(tf.metrics.recall)


In [14]:
lr_cb = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, min_delta = 0.01, patience = 3, verbose = 1)
es_cb = EarlyStopping(monitor = 'val_loss', min_delta=0.01, patience = 10, verbose = 1, restore_best_weights = True)
callbacks = [lr_cb, es_cb]

In [15]:
adam = optimizers.adam(lr=0.01)
validation_split_on_training = 0.1

In [21]:
def dense_model_generator(n_timesteps, n_features, provided_input = None):

    #Input
    if provided_input == None:
        x = Input((n_timesteps, n_features))
    else:
        x = provided_input
        
    #Dense
    dense_1 = Lambda(lambda x: K.tf.unstack(x, axis=2))(x)
    dense_2 = [Dense(20)(x) for x in dense_1]
    dense_3 = Lambda(lambda x: K.stack(x, axis=2))(dense_2)
    dense_4 = Dropout(0.1)(dense_3)
    dense_5 = Flatten()(dense_4)
    dense_6 = Dense(250, activation = 'relu')(dense_5)
    dense_7 = Dense(20, activation = 'relu')(dense_6)
    dense_8 = Dense(n_outputs, activation='softmax', name = 'dense_out')(dense_7)
    
    return x, dense_8, dense_6


def lstm_model_generator(n_timesteps, n_features, provided_input = None):
    
    #Input
    if provided_input == None:
        x = Input((n_timesteps, n_features))
    else:
        x = provided_input

    #LSTM
    lstm_1 = LSTM(100, input_shape=(n_timesteps,n_features))(x)
    lstm_2 = Dropout(0.5)(lstm_1)
    lstm_3 = Dense(100, activation='relu')(lstm_2)
    lstm_4 = Dense(n_outputs, activation='softmax', name='lstm_out')(lstm_3)
    
    return x, lstm_4, lstm_3


def hibrid_ens_generator(n_timesteps, n_features):

    dense_in, dense_out, dense_int = dense_model_generator(n_timesteps, n_features)
    lstm_in, lstm_out, lstm_int  = lstm_model_generator(n_timesteps, n_features, provided_input=dense_in)

    ens_1 = Concatenate(axis=1)([lstm_int,dense_int])
    ens_2 = Dense(n_outputs, activation='softmax')(ens_1)
    
    return dense_in, ens_2, ens_1


def hibrid_ens_2_generator(n_timesteps, n_features):

    dense_in, dense_out, dense_int = dense_model_generator(n_timesteps, n_features)
    lstm_in, lstm_out, lstm_int  = lstm_model_generator(n_timesteps, n_features, provided_input=dense_in)

    ens_1 = Concatenate(axis=1)([lstm_int,dense_int])
    ens_2 = Dense(n_outputs, activation='softmax', name = 'ens_out')(ens_1)
    
    return dense_in, [dense_out, lstm_out, ens_2], ens_1

In [27]:
model_name = 'ens_2'
tensorboard = TensorBoard(log_dir="logs/{}".format(model_name + '_' + str(time())))
# custom callbacks
lr_cb = ReduceLROnPlateau(monitor = 'val_ens_out_auc', mode = 'max',
                          factor = 0.5, min_delta = 0.01, patience = 3, verbose = 1)
es_cb = EarlyStopping(monitor = 'val_ens_out_auc',  mode = 'max',
                      min_delta=0.005, patience = 10, verbose = 1, restore_best_weights = True)
callbacks = [lr_cb, es_cb]
callbacks_ens = callbacks

ens_input, ens_output, _  = hibrid_ens_2_generator(n_timesteps, n_features)

losses = ['categorical_crossentropy'] * len(ens_output)
lossWeights = [0.2, 0.2, 0.6]

model = Model(ens_input, ens_output, name= model_name)
model.compile(loss=losses, loss_weights=lossWeights,
              optimizer='adam', metrics=['accuracy', auc_roc])
model.summary()

model.fit(x_train, 
          [y_train]*3, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=validation_split_on_training,
          verbose=True,
         callbacks = callbacks_ens)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 128, 9)       0                                            
__________________________________________________________________________________________________
lambda_15 (Lambda)              [(None, 128), (None, 0           input_8[0][0]                    
__________________________________________________________________________________________________
dense_90 (Dense)                (None, 20)           2580        lambda_15[0][0]                  
__________________________________________________________________________________________________
dense_91 (Dense)                (None, 20)           2580        lambda_15[0][1]                  
__________________________________________________________________________________________________
dense_92 (

Epoch 5/100
5881/5881 [==============================] - 4s 696us/step - loss: 0.6186 - dense_out_loss: 0.4782 - lstm_out_loss: 1.2810 - ens_out_loss: 0.4447 - dense_out_acc: 0.8407 - dense_out_auc: 0.9157 - lstm_out_acc: 0.4360 - lstm_out_auc: 0.7802 - ens_out_acc: 0.8638 - ens_out_auc: 0.9360 - val_loss: 0.7219 - val_dense_out_loss: 0.5956 - val_lstm_out_loss: 1.2055 - val_ens_out_loss: 0.6028 - val_dense_out_acc: 0.8260 - val_dense_out_auc: 0.9244 - val_lstm_out_acc: 0.4575 - val_lstm_out_auc: 0.7869 - val_ens_out_acc: 0.8382 - val_ens_out_auc: 0.9420
Epoch 6/100
5881/5881 [==============================] - 4s 689us/step - loss: 0.5085 - dense_out_loss: 0.3722 - lstm_out_loss: 1.1527 - ens_out_loss: 0.3391 - dense_out_acc: 0.8675 - dense_out_auc: 0.9315 - lstm_out_acc: 0.5169 - lstm_out_auc: 0.7940 - ens_out_acc: 0.8932 - ens_out_auc: 0.9470 - val_loss: 0.6661 - val_dense_out_loss: 0.5429 - val_lstm_out_loss: 1.1189 - val_ens_out_loss: 0.5563 - val_dense_out_acc: 0.8382 - val_dense_

5881/5881 [==============================] - 4s 699us/step - loss: 0.1896 - dense_out_loss: 0.1158 - lstm_out_loss: 0.4868 - ens_out_loss: 0.1152 - dense_out_acc: 0.9565 - dense_out_auc: 0.9821 - lstm_out_acc: 0.8128 - lstm_out_auc: 0.9115 - ens_out_acc: 0.9573 - ens_out_auc: 0.9840 - val_loss: 0.5195 - val_dense_out_loss: 0.4331 - val_lstm_out_loss: 0.8346 - val_ens_out_loss: 0.4432 - val_dense_out_acc: 0.8858 - val_dense_out_auc: 0.9826 - val_lstm_out_acc: 0.7736 - val_lstm_out_auc: 0.9135 - val_ens_out_acc: 0.8933 - val_ens_out_auc: 0.9844

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 20/100
5881/5881 [==============================] - 4s 700us/step - loss: 0.1857 - dense_out_loss: 0.1135 - lstm_out_loss: 0.4798 - ens_out_loss: 0.1117 - dense_out_acc: 0.9592 - dense_out_auc: 0.9830 - lstm_out_acc: 0.8111 - lstm_out_auc: 0.9154 - ens_out_acc: 0.9575 - ens_out_auc: 0.9847 - val_loss: 0.5093 - val_dense_out_loss: 0.4334 - val_lstm_out_loss: 0.77

In [16]:
model_name = 'LSTM'
tensorboard = TensorBoard(log_dir="logs/{}".format(model_name + '_' + str(time())))
callbacks_lstm = callbacks + [tensorboard]

lstm_input, lstm_output, _  = lstm_model_generator(n_timesteps, n_features)

model = Model(lstm_input, lstm_output, name= model_name )
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, 
          y_train, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=validation_split_on_training,
          verbose=True,
         callbacks = callbacks_lstm)


Train on 5881 samples, validate on 1471 samples
Epoch 1/100
5881/5881 [==============================] - 5s 777us/step - loss: 1.7172 - acc: 0.3226 - val_loss: 1.5621 - val_acc: 0.4439
Epoch 2/100
5881/5881 [==============================] - 4s 640us/step - loss: 1.3972 - acc: 0.4023 - val_loss: 1.3558 - val_acc: 0.4521
Epoch 3/100
5881/5881 [==============================] - 4s 642us/step - loss: 1.2073 - acc: 0.4727 - val_loss: 1.1386 - val_acc: 0.4691
Epoch 4/100
5881/5881 [==============================] - 4s 643us/step - loss: 1.0849 - acc: 0.5195 - val_loss: 1.0738 - val_acc: 0.5452
Epoch 5/100
5881/5881 [==============================] - 4s 642us/step - loss: 1.0175 - acc: 0.5552 - val_loss: 1.0296 - val_acc: 0.5948
Epoch 6/100
5881/5881 [==============================] - 4s 639us/step - loss: 0.9429 - acc: 0.5967 - val_loss: 0.9388 - val_acc: 0.6023
Epoch 7/100
5881/5881 [==============================] - 4s 641us/step - loss: 0.8041 - acc: 0.6524 - val_loss: 0.8692 - val_acc: 

### Dense network approach

some comments about this architecture:

- Note that the unstack dim is the feature dimension

In [17]:
model_name = 'dense'
tensorboard = TensorBoard(log_dir="logs/{}".format(model_name + '_' + str(time())))
callbacks_dense = callbacks + [tensorboard]

#generate model
dense_input, dense_output , _ = dense_model_generator(n_timesteps, n_features)
model = Model(dense_input, dense_output, name = model_name)

#compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(x_train, 
          y_train, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=validation_split_on_training,
          verbose=True,
         callbacks = callbacks_dense)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 9)       0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               [(None, 128), (None, 0           input_2[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 20)           2580        lambda_1[0][0]                   
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 20)           2580        lambda_1[0][1]                   
__________________________________________________________________________________________________
dense_4 (D

# Hybrid ensemble

In [18]:
model_name = 'ens'
tensorboard = TensorBoard(log_dir="logs/{}".format(model_name + '_' + str(time())))
callbacks_ens = callbacks + [tensorboard]

ens_input, ens_output, _  = hibrid_ens_generator(n_timesteps, n_features)


model = Model(ens_input, ens_output, name= model_name)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(x_train, 
          y_train, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=validation_split_on_training,
          verbose=True,
         callbacks = callbacks_ens)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 9)       0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               [(None, 128), (None, 0           input_3[0][0]                    
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 20)           2580        lambda_3[0][0]                   
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 20)           2580        lambda_3[0][1]                   
__________________________________________________________________________________________________
dense_16 (